In [24]:
import json
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [29]:
def obter_dados(fonte):
    if fonte == 'json':
        with open('cursos.json') as f:
            dados = json.load(f)
    elif fonte == 'webhook':
        url = 'https://exemplo.com/api/dados'
        response = requests.get(url)
        dados = response.json()
    else:
        raise ValueError('Fonte de dados inválida')
    return dados

In [30]:
fonte = input('Digite a fonte dos dados (json ou webhook): ')
dados = obter_dados(fonte)

In [31]:
df = pd.DataFrame(columns=['id', 'nome', 'idade', 'curso_nome', 'curso_categoria', 'curso_avaliacao', 'amigo_id', 'amigo_nome', 'amigo_curso_nome', 'amigo_curso_avaliacao'])

In [35]:
rows = []
for usuario in data['usuarios']:
    for curso in usuario['cursos']:
        row = {
            'id': usuario['id'],
            'nome': usuario['nome'],
            'idade': usuario['idade'],
            'curso_nome': curso['nome'],
            'curso_categoria': curso['categoria'],
            'curso_avaliacao': curso['avaliacao'],
            'amigo_id': None,
            'amigo_nome': None,
            'amigo_curso_nome': None,
            'amigo_curso_avaliacao': None
        }
        rows.append(row)

        for amigo in usuario['amigos']:
            for curso in amigo['cursos']:
                row = {
                    'id': usuario['id'],
                    'nome': usuario['nome'],
                    'idade': usuario['idade'],
                    'curso_nome': None,
                    'curso_categoria': None,
                    'curso_avaliacao': None,
                    'amigo_id': amigo['id'],
                    'amigo_nome': amigo['nome'],
                    'amigo_curso_nome': curso['nome'],
                    'amigo_curso_avaliacao': curso['avaliacao']
                }
                rows.append(row)

df = pd.DataFrame(rows)

In [36]:
df.drop_duplicates(inplace=True)

In [37]:
cursos_agrupados = df.groupby(['curso_nome', 'curso_categoria'])['curso_avaliacao'].mean().reset_index()

In [38]:
cursos_agrupados.rename(columns={'curso_avaliacao': 'curso_media_avaliacao'}, inplace=True)

In [40]:
df = df.merge(cursos_agrupados, on=['curso_nome', 'curso_categoria'], how='left')

In [41]:
df.sort_values(['nome', 'curso_media_avaliacao'], ascending=[True, False], inplace=True)

In [42]:
print(df.head())

   id   nome  idade                                      curso_nome   
0   1  JoÃ£o     25         IntroduÃ§Ã£o Ã  ProgramaÃ§Ã£o em Python  \
9   1  JoÃ£o     25                          InglÃªs IntermediÃ¡rio   
8   1  JoÃ£o     25  Desenvolvimento Web com HTML, CSS e JavaScript   
1   1  JoÃ£o     25                                            None   
2   1  JoÃ£o     25                                            None   

  curso_categoria  curso_avaliacao  amigo_id amigo_nome    amigo_curso_nome   
0   ProgramaÃ§Ã£o              4.5       NaN       None                None  \
9         Idiomas              4.0       NaN       None                None   
8   ProgramaÃ§Ã£o              3.5       NaN       None                None   
1            None              NaN       5.0      Maria  FinanÃ§as Pessoais   
2            None              NaN       5.0      Maria   Marketing Digital   

   amigo_curso_avaliacao  curso_media_avaliacao  
0                    NaN                    4.5 